#**Revisão de limpeza de dados**
Importando a base de clientes de um banco com o resultado do pagamento ou não de um empréstimo

In [29]:
import pandas as pd

In [30]:
base = pd.read_excel("ChavesClientes.xlsx",sheet_name = "base")

In [31]:
base.head()
#visualizar as 5 primeiras cedulas da base

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0



## Verificando a cardinalidade desses dados ( quantos valore diferentes existem em cada categoria.

In [32]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()


Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### Ao analisar essa base, percebemos que existe uma alta cardinalidade e por isso precisamos tratar esses dados

- Ao mapear com o responsável pela área, ele informou o que representa cada coluna:
    - ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    

Começamos a limpeza separando as informações da coluna ChaveSituação.
o primeiro passo será separar o estado civil do cliente o que é o ultimo dado sa chave.


In [33]:
texto = "32FC"
texto  [-1]

'C'

In [34]:
base['EstadoCivil'] = base.ChaveSituacao.str[-1]

In [35]:
base.head()


,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,C
1,2,25MV,AAmarelo,Black,1,V
2,3,27MV,B-Amarelo,Basic-Beta,1,V
3,4,26FD,BAmarelo,Black,0,D
4,5,26FD,C-Amarelo,Black,0,D


Agora cou criar uma coluna para a idade do cliente


In [36]:
base['Idade'] = base.ChaveSituacao.str[:2]
base.head()


,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,EstadoCivil,Idade
0,1,32FC,Ccinza,Basic-Alpha,1,C,32
1,2,25MV,AAmarelo,Black,1,V,25
2,3,27MV,B-Amarelo,Basic-Beta,1,V,27
3,4,26FD,BAmarelo,Black,0,D,26
4,5,26FD,C-Amarelo,Black,0,D,26


agora vou criar uma coluna para o genero do cliente

In [37]:
base['Genero'] = base.ChaveSituacao.str[2:3]
base.head()


,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,EstadoCivil,Idade,Genero
0,1,32FC,Ccinza,Basic-Alpha,1,C,32,F
1,2,25MV,AAmarelo,Black,1,V,25,M
2,3,27MV,B-Amarelo,Basic-Beta,1,V,27,M
3,4,26FD,BAmarelo,Black,0,D,26,F
4,5,26FD,C-Amarelo,Black,0,D,26,F


Agora que ja separaei os dados da coluna ChaveSituação, vamos para a Cat de Cliente. Para isso vou usar o split o qual separa strings por um delimitador  e cria duas listas diferentes.
CatCliente: formado por:
Categoria do cartão: qual o tipo de cartão do cliente:
Basic
Black
Platinum

In [38]:
texto = 'Basic-Alpha'
texto.split('-')

['Basic', 'Alpha']

In [39]:
base['Categoria'] = base.CatCliente.str.split('-').str.get(0)
base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)
base.head()



,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,EstadoCivil,Idade,Genero,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,C,32,F,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,V,25,M,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,V,27,M,Basic,Beta
3,4,26FD,BAmarelo,Black,0,D,26,F,Black,NaN
4,5,26FD,C-Amarelo,Black,0,D,26,F,Black,NaN


Caso a gente não consiga separar pela posição do texto ou por um delimitador, ainda temos a opção de usar o Regex

In [40]:
import re

In [43]:
base["Risco"] =base.ClassRisco.apply(lambda x: re.findall("^[A-Z][^A-Za-z]?",x)[0])
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,EstadoCivil,Idade,Genero,Categoria,CatVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,C,32,F,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,V,25,M,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,V,27,M,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,D,26,F,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,D,26,F,Black,NaN,C-


In [44]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   EstadoCivil    20 non-null     object
 6   Idade          20 non-null     object
 7   Genero         20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


Transformando a coluna 'Idade' em numérico

In [46]:
base['Idade'] = pd.to_numeric(base.Idade)
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   EstadoCivil    20 non-null     object
 6   Idade          20 non-null     int64 
 7   Genero         20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


Criar uma nova base com somente as colunas novas ou seja os dados limpos

In [49]:
base_limpa = base[["Idade","Genero","EstadoCivil","Categoria","CatVIP","Risco","Pagamento"]]
display(base_limpa)


,Idade,Genero,EstadoCivil,Categoria,CatVIP,Risco,Pagamento
0,32,F,C,Basic,Alpha,C,1
1,25,M,V,Black,NaN,A,1
2,27,M,V,Basic,Beta,B-,1
3,26,F,D,Black,NaN,B,0
4,26,F,D,Black,NaN,C-,0
5,28,F,C,Platinum,Alpha,C-,0
6,27,M,D,Platinum,Beta,A-,1
7,31,M,D,Basic,NaN,C-,0
8,28,F,S,Black,NaN,A-,1
9,31,M,V,Platinum,NaN,C+,1


In [50]:
base.groupby(["Pagamento","Categoria"])["Pagamento"].count()

Pagamento  Categoria
0          Basic        3
           Black        3
           Platinum     1
1          Basic        3
           Black        4
           Platinum     6
Name: Pagamento, dtype: int64